# 1.Scraping the pokedex

In [51]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
pokedex_url = 'https://pokemondb.net/pokedex/all'

In [52]:
pokedex = requests.get(pokedex_url)
# Response code is returned

In [53]:
pokedex_soup = BeautifulSoup(pokedex.text, 'html.parser')
pokedex_soup.text[:500]

"\n\n\n\nPokémon Pokédex: list of Pokémon with stats | Pokémon Database\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t(function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){\n\t\t(i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),\n\t\tm=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n\t\t})(window,document,'script','https://www.google-analytics.com/analytics.js','ga');\n\t\tga('create', 'UA-1974891-7', 'auto');\n\t\tga('send', 'pageview');\n\t\n\n.main-menu-item:hover > "

In [54]:
pokedex_list = pokedex_soup.find('table', {'id':'pokedex'}, {'class':'data-talbe block-wide'})
pokedex_list.text[:500]

'\n\n\n# Name Type Total HP Attack Defense Sp. Atk Sp. Def Speed \n\n\n\n001 BulbasaurGrass Poison\n318\n45\n49\n49\n65\n65\n45\n\n\n002 IvysaurGrass Poison\n405\n60\n62\n63\n80\n80\n60\n\n\n003 VenusaurGrass Poison\n525\n80\n82\n83\n100\n100\n80\n\n\n003 Venusaur Mega VenusaurGrass Poison\n625\n80\n100\n123\n122\n120\n80\n\n\n004 CharmanderFire \n309\n39\n52\n43\n60\n50\n65\n\n\n005 CharmeleonFire \n405\n58\n64\n58\n80\n65\n80\n\n\n006 CharizardFire Flying\n534\n78\n84\n78\n109\n85\n100\n\n\n006 Charizard Mega Charizard XFire Dragon\n634\n78\n130\n111\n130\n85\n100\n\n\n006 Chariz'

In [55]:
row_list = pokedex_list.findAll('tr')
len(row_list)

1035

There are 1035 rows 

Step 1.2

In [56]:
first_row = row_list[1]
first_row

<tr>
<td class="cell-num cell-fixed" data-sort-value="1"><span class="infocard-cell-img"><span class="img-fixed icon-pkmn" data-alt="Bulbasaur icon" data-src="https://img.pokemondb.net/sprites/sword-shield/icon/bulbasaur.png"></span></span><span class="infocard-cell-data">001</span></td> <td class="cell-name"><a class="ent-name" href="/pokedex/bulbasaur" title="View pokedex for #001 Bulbasaur">Bulbasaur</a></td><td class="cell-icon"><a class="type-icon type-grass" href="/type/grass">Grass</a><br/> <a class="type-icon type-poison" href="/type/poison">Poison</a></td>
<td class="cell-total">318</td>
<td class="cell-num">45</td>
<td class="cell-num">49</td>
<td class="cell-num">49</td>
<td class="cell-num">65</td>
<td class="cell-num">65</td>
<td class="cell-num">45</td>
</tr>

Step 1.2

In [57]:
first_row_name = first_row.find(class_="ent-name").text
print("The name of the pokemon:", first_row_name)
first_row_url = first_row.find('a').get("href")
print("The url to the pokemon's page:", first_row_url)
print("The type or types:", first_row.find(class_ = 'cell-icon').text)
print("The total points:", first_row.find(class_ = 'cell-total').text)

The name of the pokemon: Bulbasaur
The url to the pokemon's page: /pokedex/bulbasaur
The type or types: Grass Poison
The total points: 318


In [58]:
pokemon = []
for div in first_row.find_all(class_ = 'cell-num'):
    pokemon.append(div.text)
print("[ID Number, HP, Attack, Defense, Sp. Atk, Sp. Def, Speed]: ", pokemon)

[ID Number, HP, Attack, Defense, Sp. Atk, Sp. Def, Speed]:  ['001', '45', '49', '49', '65', '65', '45']


### Define get_pokemon_df  

In [59]:
def get_pokemon_df(pokedex_row, df):
    first_row_name = pokedex_row.find(class_="ent-name").text
    first_row_url = pokedex_row.find('a').get("href")
    first_row_type = pokedex_row.find(class_ = 'cell-icon').text
    first_row_points = pokedex_row.find(class_ = 'cell-total').text
    num = []
    for div in pokedex_row.find_all(class_ = 'cell-num'):
        num.append(div.text)
    temp_row = np.array([[num[0], first_row_name, first_row_url, first_row_type, 
                          first_row_points, num[1],num[2],num[3], num[4], num[5],num[6]]])
    temp_df = pd.DataFrame(temp_row, columns = ['ID Number', 'Name', 'Url', 'Type', 'Total points', 'HP',
                                                   'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed'])

    return df.append(temp_df)

In [60]:
row_list_withoutFirst = row_list[1:]
df =  pd.DataFrame()
for row in row_list_withoutFirst:
    df = get_pokemon_df(row, df)

In [61]:
df = df.reset_index(drop = True)
df.head(10)

,ID Number,Name,Url,Type,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,001,Bulbasaur,/pokedex/bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,002,Ivysaur,/pokedex/ivysaur,Grass Poison,405,60,62,63,80,80,60
2,003,Venusaur,/pokedex/venusaur,Grass Poison,525,80,82,83,100,100,80
3,003,Venusaur,/pokedex/venusaur,Grass Poison,625,80,100,123,122,120,80
4,004,Charmander,/pokedex/charmander,Fire,309,39,52,43,60,50,65
5,005,Charmeleon,/pokedex/charmeleon,Fire,405,58,64,58,80,65,80
6,006,Charizard,/pokedex/charizard,Fire Flying,534,78,84,78,109,85,100
7,006,Charizard,/pokedex/charizard,Fire Dragon,634,78,130,111,130,85,100
8,006,Charizard,/pokedex/charizard,Fire Flying,634,78,104,78,159,115,100
9,007,Squirtle,/pokedex/squirtle,Water,314,44,48,65,50,64,43


# 2. Cleaning the Pokedex
* Add column names 
* Convert strings to numeric 
* Create dummy variables
* Remove duplicate values 
* Create a sample of the pokemon

In [62]:
df.dtypes #check original data types

ID Number       object
Name            object
Url             object
Type            object
Total points    object
HP              object
Attack          object
Defense         object
Sp. Atk         object
Sp. Def         object
Speed           object
dtype: object

In [63]:
df['Total points'] = pd.to_numeric(df['Total points'])
df['HP'] =  pd.to_numeric(df['HP'])
df['Attack'] = pd.to_numeric(df['Attack'])
df['Defense'] = pd.to_numeric(df['Defense'])
df['Sp. Atk'] = pd.to_numeric(df['Sp. Atk'])
df['Sp. Def'] = pd.to_numeric(df['Sp. Def'])
df['Speed'] = pd.to_numeric(df['Speed'])

In [64]:
df2 = df.copy()

In [65]:
df2['Type2'] = df2['Type'].str.split().str[1]
df2['Type'] = df2['Type'].str.split().str[0]
df2.head(10)   #split types for each pokemon in df2

,ID Number,Name,Url,Type,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type2
0,001,Bulbasaur,/pokedex/bulbasaur,Grass,318,45,49,49,65,65,45,Poison
1,002,Ivysaur,/pokedex/ivysaur,Grass,405,60,62,63,80,80,60,Poison
2,003,Venusaur,/pokedex/venusaur,Grass,525,80,82,83,100,100,80,Poison
3,003,Venusaur,/pokedex/venusaur,Grass,625,80,100,123,122,120,80,Poison
4,004,Charmander,/pokedex/charmander,Fire,309,39,52,43,60,50,65,NaN
5,005,Charmeleon,/pokedex/charmeleon,Fire,405,58,64,58,80,65,80,NaN
6,006,Charizard,/pokedex/charizard,Fire,534,78,84,78,109,85,100,Flying
7,006,Charizard,/pokedex/charizard,Fire,634,78,130,111,130,85,100,Dragon
8,006,Charizard,/pokedex/charizard,Fire,634,78,104,78,159,115,100,Flying
9,007,Squirtle,/pokedex/squirtle,Water,314,44,48,65,50,64,43,NaN


In [66]:
len(df2['Type'].unique()) #check 18 types
dummy = pd.get_dummies(df2['Type'])
dummy2 = pd.get_dummies(df2['Type2'])

In [67]:
res = pd.concat([dummy, dummy2], axis=1)
res.head()

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Remove duplicate values

In [68]:
df.drop_duplicates(subset ="Url", keep = 'first', inplace = True) 
df.head(10)  #remove duplicate values of pokemon based on the URL

,ID Number,Name,Url,Type,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,001,Bulbasaur,/pokedex/bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,002,Ivysaur,/pokedex/ivysaur,Grass Poison,405,60,62,63,80,80,60
2,003,Venusaur,/pokedex/venusaur,Grass Poison,525,80,82,83,100,100,80
4,004,Charmander,/pokedex/charmander,Fire,309,39,52,43,60,50,65
5,005,Charmeleon,/pokedex/charmeleon,Fire,405,58,64,58,80,65,80
6,006,Charizard,/pokedex/charizard,Fire Flying,534,78,84,78,109,85,100
9,007,Squirtle,/pokedex/squirtle,Water,314,44,48,65,50,64,43
10,008,Wartortle,/pokedex/wartortle,Water,405,59,63,80,65,80,58
11,009,Blastoise,/pokedex/blastoise,Water,530,79,83,100,85,105,78
13,010,Caterpie,/pokedex/caterpie,Bug,195,45,30,35,20,20,45


In [69]:
print(len(df)) #Print the number of rows in the deduplicated dataset
df = df.reset_index(drop = True) #reset the index

893


### Create sample of the pokemon.

In [74]:
sample_list = [None for i in range(len(df))]
for i in range(809):
    if (i+1) % 4 == 0 : 
        sample_list[i] = 1
    else:
        sample_list[i] = 0

In [75]:
df['sample'] = sample_list
df.head(10)

,ID Number,Name,Url,Type,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,sample
0,001,Bulbasaur,/pokedex/bulbasaur,Grass Poison,318,45,49,49,65,65,45,0.0
1,002,Ivysaur,/pokedex/ivysaur,Grass Poison,405,60,62,63,80,80,60,0.0
2,003,Venusaur,/pokedex/venusaur,Grass Poison,525,80,82,83,100,100,80,0.0
3,004,Charmander,/pokedex/charmander,Fire,309,39,52,43,60,50,65,1.0
4,005,Charmeleon,/pokedex/charmeleon,Fire,405,58,64,58,80,65,80,0.0
5,006,Charizard,/pokedex/charizard,Fire Flying,534,78,84,78,109,85,100,0.0
6,007,Squirtle,/pokedex/squirtle,Water,314,44,48,65,50,64,43,0.0
7,008,Wartortle,/pokedex/wartortle,Water,405,59,63,80,65,80,58,1.0
8,009,Blastoise,/pokedex/blastoise,Water,530,79,83,100,85,105,78,0.0
9,010,Caterpie,/pokedex/caterpie,Bug,195,45,30,35,20,20,45,0.0


# 3. Scraping Individual Pages
* Scrape the main image for Bulbasaur 
* Extract the location table

In [77]:
from IPython.display import Image
import urllib.request
import os

In [78]:
Image(url='https://img.pokemondb.net/artwork/bulbasaur.jpg')

## Extract the location table

In [79]:
url = 'https://pokemondb.net/pokedex/bulbasaur'

In [80]:
tables = pd.read_html(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text)

In [81]:
df_BulbasaurLocations = tables[15]
df_BulbasaurLocations

,0,1
0,RedBlue,Pallet Town
1,Yellow,Cerulean City
2,GoldSilverCrystal,Trade/migrate from another game
3,RubySapphire,Trade/migrate from another game
4,FireRedLeafGreen,Pallet Town
5,Emerald,Trade/migrate from another game
6,DiamondPearlPlatinum,Trade/migrate from another game
7,HeartGoldSoulSilver,Pallet Town
8,BlackWhiteBlack 2White 2,Trade/migrate from another game
9,XY,Lumiose City


## Transpose the DataFrame

In [82]:
df_BulbasaurLocations_transpose = df_BulbasaurLocations.transpose()
new_header = df_BulbasaurLocations_transpose.iloc[0] #grab the first row for the header
df_BulbasaurLocations_transpose = df_BulbasaurLocations_transpose[1:] #take the data less the header row
df_BulbasaurLocations_transpose.columns = new_header
df_BulbasaurLocations_transpose

,RedBlue,Yellow,GoldSilverCrystal,RubySapphire,FireRedLeafGreen,Emerald,DiamondPearlPlatinum,HeartGoldSoulSilver,BlackWhiteBlack 2White 2,XY,Omega RubyAlpha Sapphire,SunMoon,Ultra SunUltra Moon,Let's Go PikachuLet's Go Eevee,SwordShield
1,Pallet Town,Cerulean City,Trade/migrate from another game,Trade/migrate from another game,Pallet Town,Trade/migrate from another game,Trade/migrate from another game,Pallet Town,Trade/migrate from another game,Lumiose City,Trade/migrate from another game,Trade/migrate from another game,Route 2,"Cerulean City, Viridian Forest",Location data not yet available


## Extracting the location information for Pokemons in the X and Y games 

In [83]:
new_series = df['Url'][df['sample'] == 1] #df is a pokemon dataframe with sample
len(new_series)

202

In [86]:
def get_location(Url):
    url = 'https://pokemondb.net'+ Url
    tables = pd.read_html(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text)
    df_BulbasaurLocations = tables[-2]
    df_BulbasaurLocations_transpose = df_BulbasaurLocations.transpose()
    new_header = df_BulbasaurLocations_transpose.iloc[0] #grab the first row for the header
    df_BulbasaurLocations_transpose = df_BulbasaurLocations_transpose[1:] #take the data less the header row
    df_BulbasaurLocations_transpose.columns = new_header
    try:
        ans = df_BulbasaurLocations_transpose['XY'].values[0]
        return ans
    except KeyError:
        return 
    

In [ ]:
XY_location = []
URL_Total = []
for u in new_series:
    location = get_location(u)
    if location != None:
        XY_location.append(location)
        URL_Total.append(u)

In [155]:
ans_df = pd.DataFrame(list(zip(URL_Total, XY_location)), 
               columns =['Url', 'XY location']) 

In [238]:
ans_df.head(10)

,Url,XY location
0,/pokedex/charmander,Lumiose City
1,/pokedex/wartortle,Evolve Squirtle
2,/pokedex/butterfree,Evolve Caterpie/Metapod
3,/pokedex/pidgey,"Route 2, 3"
4,/pokedex/raticate,Trade/migrate from another game
5,/pokedex/arbok,Route 19
6,/pokedex/sandslash,"Route 18, Terminus Cave"
7,/pokedex/nidoran-m,Route 11
8,/pokedex/clefable,Trade/migrate from another game
9,/pokedex/gloom,Evolve Oddish


# Analysis 

## Which type has the highest and lowest average attack? Average defense?
* Fighting has the highest average attack.
* Fairy has the lowest average attack.
* Steel has the highest average defense.
* Normal has the lowest average defense.

## Which location has the highest average total point score?
* Sea Spirit's Den and Roaming Kalos
* Total points is 580.

In [247]:
df2.head() # df2 is a dataframe which still have repeat Url

,ID Number,Name,Url,Type,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type2
0,001,Bulbasaur,/pokedex/bulbasaur,Grass,318,45,49,49,65,65,45,Poison
1,002,Ivysaur,/pokedex/ivysaur,Grass,405,60,62,63,80,80,60,Poison
2,003,Venusaur,/pokedex/venusaur,Grass,525,80,82,83,100,100,80,Poison
3,003,Venusaur,/pokedex/venusaur,Grass,625,80,100,123,122,120,80,Poison
4,004,Charmander,/pokedex/charmander,Fire,309,39,52,43,60,50,65,NaN


In [188]:
unique_type = df2['Type'].unique()

In [191]:
attack_list = []
defend_list = []
for ty in unique_type:
    table = df2[(df2['Type'] == ty) | (df2['Type2'] == ty)]
    attack_list.append(table['Attack'].mean())
    defend_list.append(table['Defense'].mean())

In [194]:
data = {'Type': unique_type, 'Average attack': attack_list, 'Average defense':defend_list}
type_table = pd.DataFrame.from_dict(data)
type_table

,Type,Average attack,Average defense
0,Grass,75.165138,73.752294
1,Fire,83.527027,70.324324
2,Water,73.965035,75.405594
3,Bug,72.734940,72.168675
4,Normal,74.555556,60.256410
5,Dark,95.066667,69.216667
6,Poison,70.521127,64.619718
7,Electric,73.741935,69.693548
8,Ground,92.520000,86.906667
9,Ice,82.340909,77.386364


In [207]:
print(type_table[type_table['Average attack'] == type_table['Average attack'].max()]['Type'])
print(type_table[type_table['Average attack'] == type_table['Average attack'].min()]['Type'])
print(type_table[type_table['Average defense'] == type_table['Average defense'].max()]['Type'])
print(type_table[type_table['Average defense'] == type_table['Average defense'].min()]['Type'])

11    Fighting
Name: Type, dtype: object
10    Fairy
Name: Type, dtype: object
16    Steel
Name: Type, dtype: object
4    Normal
Name: Type, dtype: object


In [214]:
ans_df.head() # location DataFrame created in Step 3.4

,Url,XY location
0,/pokedex/charmander,Lumiose City
1,/pokedex/wartortle,Evolve Squirtle
2,/pokedex/butterfree,Evolve Caterpie/Metapod
3,/pokedex/pidgey,"Route 2, 3"
4,/pokedex/raticate,Trade/migrate from another game


In [215]:
df2.head() # df2 is a dataframe which still have repeat Url

,ID Number,Name,Url,Type,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type2
0,001,Bulbasaur,/pokedex/bulbasaur,Grass,318,45,49,49,65,65,45,Poison
1,002,Ivysaur,/pokedex/ivysaur,Grass,405,60,62,63,80,80,60,Poison
2,003,Venusaur,/pokedex/venusaur,Grass,525,80,82,83,100,100,80,Poison
3,003,Venusaur,/pokedex/venusaur,Grass,625,80,100,123,122,120,80,Poison
4,004,Charmander,/pokedex/charmander,Fire,309,39,52,43,60,50,65,NaN


In [217]:
Join_table = pd.merge(df2, ans_df, on='Url', how='right')
#Join the pokedex data to the  location DataFrame created in Step 3.4.
Join_table.head()

,ID Number,Name,Url,Type,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type2,XY location
0,004,Charmander,/pokedex/charmander,Fire,309,39,52,43,60,50,65,NaN,Lumiose City
1,008,Wartortle,/pokedex/wartortle,Water,405,59,63,80,65,80,58,NaN,Evolve Squirtle
2,012,Butterfree,/pokedex/butterfree,Bug,395,60,45,50,90,80,70,Flying,Evolve Caterpie/Metapod
3,016,Pidgey,/pokedex/pidgey,Normal,251,40,45,40,35,35,56,Flying,"Route 2, 3"
4,020,Raticate,/pokedex/raticate,Normal,413,55,81,60,50,70,97,NaN,Trade/migrate from another game


In [237]:
Location_points_mean_series = Join_table.groupby(by='XY location')['Total points'].mean()
Location_points_mean_df = Location_points_mean_series.to_frame()
Location_points_mean_df.head(10)

,Total points
XY location,
Ambrette Town,355.5
Breed Electrode,330.0
Breed Gurdurr/Conkeldurr,305.0
Breed Hariyama,237.0
Breed Haunter/Gengar,310.0
Breed Piloswine/Mamoswine,250.0
Breed Ursaring,330.0
Breed Watchog,255.0
Cyllage City,560.0


In [235]:
Location_points_mean_df[Location_points_mean_df['Total points'] == Location_points_mean_df['Total points'].max()]

,Total points
XY location,
"Sea Spirit's Den, Roaming Kalos",580.0
